# Day 12 Reading Journal

This journal includes several required exercises, but it is meant to encourage active reading more generally.  You should use the journal to take detailed notes, catalog questions, and explore the content from Think Python deeply.

Reading: Review Think Python Chapter 18

**Due: Monday, March 6 at 12 noon**



## [Chapter 18](http://www.greenteapress.com/thinkpython2/html/thinkpython2019.html)

The exercises writing class methods in this chapter have a large amount of supporting code. It may be more natural for you to do your development at the command line and you are welcome to, but please paste your solutions back in the notebook when you're done.


### Exercise 3  

Write a method belonging to `Deck` called `deal_hands` that takes two parameters, the number of hands and the number of cards per hand, and that creates new `Hand` objects, deals the appropriate number of cards per hand, and returns a list of `Hand` objects.

In [1]:
"""This module contains a code example related to

Think Python, 2nd Edition
by Allen Downey
http://thinkpython2.com

Copyright 2015 Allen Downey

License: http://creativecommons.org/licenses/by/4.0/
"""

from __future__ import print_function, division

import random


class Card:
    """Represents a standard playing card.
    
    Attributes:
      suit: integer 0-3
      rank: integer 1-13
    """

    suit_names = ["Clubs", "Diamonds", "Hearts", "Spades"]
    rank_names = [None, "Ace", "2", "3", "4", "5", "6", "7", 
              "8", "9", "10", "Jack", "Queen", "King"]

    def __init__(self, suit=0, rank=2):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        """Returns a human-readable string representation."""
        return '%s of %s' % (Card.rank_names[self.rank],
                             Card.suit_names[self.suit])

    def __eq__(self, other):
        """Checks whether self and other have the same rank and suit.

        returns: boolean
        """
        return self.suit == other.suit and self.rank == other.rank

    def __lt__(self, other):
        """Compares this card to other, first by suit, then rank.

        returns: boolean
        """
        t1 = self.suit, self.rank
        t2 = other.suit, other.rank
        return t1 < t2


class Deck:
    """Represents a deck of cards.

    Attributes:
      cards: list of Card objects.
    """
    
    def __init__(self):
        """Initializes the Deck with 52 cards.
        """
        self.cards = []
        for suit in range(4):
            for rank in range(1, 14):
                card = Card(suit, rank)
                self.cards.append(card)

    def __str__(self):
        """Returns a string representation of the deck.
        """
        res = []
        for card in self.cards:
            res.append(str(card))
        return '\n'.join(res)

    def add_card(self, card):
        """Adds a card to the deck.

        card: Card
        """
        self.cards.append(card)

    def remove_card(self, card):
        """Removes a card from the deck or raises exception if it is not there.
        
        card: Card
        """
        self.cards.remove(card)

    def pop_card(self, i=-1):
        """Removes and returns a card from the deck.

        i: index of the card to pop; by default, pops the last card.
        """
        return self.cards.pop(i)

    def shuffle(self):
        """Shuffles the cards in this deck."""
        random.shuffle(self.cards)

    def sort(self):
        """Sorts the cards in ascending order."""
        self.cards.sort()

    def move_cards(self, hand, num):
        """Moves the given number of cards from the deck into the Hand.

        hand: destination Hand object
        num: integer number of cards to move
        """
        for i in range(num):
            hand.add_card(self.pop_card())


class Hand(Deck):
    """Represents a hand of playing cards."""
    
    def __init__(self, label=''):
        self.cards = []
        self.label = label


def find_defining_class(obj, method_name):
    """Finds and returns the class object that will provide 
    the definition of method_name (as a string) if it is
    invoked on obj.

    obj: any python object
    method_name: string method name
    """
    for ty in type(obj).mro():
        if method_name in ty.__dict__:
            return ty
    return None


if __name__ == '__main__':
    deck = Deck()
    deck.shuffle()

    hand = Hand()
    print(find_defining_class(hand, 'shuffle'))

    deck.move_cards(hand, 5)
    hand.sort()
    print(hand)

<class '__main__.Deck'>
2 of Clubs
Queen of Clubs
3 of Diamonds
2 of Hearts
5 of Spades


### Exercise 6  

**Note:** Jupyter notebooks can access code in other cells, so as long as you have run the cell above then the `PokerHand` class above will be able to reference your previous definition of the `Hand` class.


The following are the possible hands in poker, in increasing order of value (and decreasing order of probability):

 1. **pair:** two cards with the same rank 
 2. **two pair:** two pairs of cards with the same rank 
 3. **three of a kind:** three cards with the same rank 
 4. **straight:** five cards with ranks in sequence (aces can be high or low, so Ace-2-3-4-5 is a straight and so is 10-Jack-Queen-King-Ace, but Queen-King-Ace-2-3 is not.) 
 5. **flush:** five cards with the same suit 
 6. **full house:** three cards with one rank, two cards with another 
 7. **four of a kind:** four cards with the same rank 
 8. **straight flush:** five cards in sequence (as defined above) and with the same suit 

The goal of these exercises is to estimate the probability of drawing these various hands. You can go as far as you like with this exercise, but you should at least attempt parts 2 and 3.

 1. If you run the code below, it deals seven 7-card poker hands and checks to see if any of them contains a flush. Read this code carefully before you go on.
 2. Add methods to `PokerHand` named `has_pair`, `has_twopair`, etc. that return `True` or `False` according to whether or not the hand meets the relevant criteria. Your code should work correctly for “hands” that contain any number of cards (although 5 and 7 are the most common sizes).
 3. Write a method named `classify` that figures out the highest-value classification for a hand and sets the label attribute accordingly. For example, a 7-card hand might contain a flush and a pair; it should be labeled “flush”.
 4. When you are convinced that your classification methods are working, the next step is to estimate the probabilities of the various hands. Write a function below that shuffles a deck of cards, divides it into hands, classifies the hands, and counts the number of times various classifications appear.
 5. Print a table of the classifications and their probabilities. Run your program with larger and larger numbers of hands until the output values converge to a reasonable degree of accuracy. Compare your results to the values at http://en.wikipedia.org/wiki/Hand_rankings.

Allen's solution:http://greenteapress.com/thinkpython2/code/PokerHandSoln.py. 

In [31]:
import doctest

class PokerHand(Hand):
    """Represents a poker hand."""

    def suit_hist(self):
        """Builds a histogram of the suits that appear in the hand.

        Stores the result in attribute suits.
        """
        self.suits = {}
        for card in self.cards:
            self.suits[card.suit] = self.suits.get(card.suit, 0) + 1

    def has_flush(self):
        """Returns True if the hand has a flush, False otherwise.

        Note that this works correctly for hands with more than 5 cards.
        """
        self.suit_hist()
        for val in self.suits.values():
            if val >= 5:
                return True
        return False

    def has_pair(self):
        """ Returns True if the hand has two cards of the same rank, False otherwise 
        
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,2), Card(2,2), Card(1,6), Card(3,7), Card(2,4), Card(1,3), Card(0,5)])
        >>> hand.has_pair()
        True
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,2), Card(2,9), Card(1,6), Card(3,7), Card(2,4), Card(1,3), Card(0,5)])
        >>> hand.has_pair()
        False
        """
        for i1 in range(len(self.cards)):
            for i2 in range(i1 + 1, len(self.cards)):
                if self.cards[i1].rank == self.cards[i2].rank:
                    return True
        return False

    def has_two_pair(self):
        """ Returns True if the hand has two pairs of cards of the same rank, False otherwise 
        
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,2), Card(2,2), Card(1,6), Card(3,3), Card(2,4), Card(1,3), Card(0,5)])
        >>> hand.has_two_pair()
        True
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,4), Card(2,2), Card(1,6), Card(3,3), Card(2,4), Card(1,1), Card(0,5)])
        >>> hand.has_two_pair()
        False
        """
        pair_count = 0
        for i1 in range(len(self.cards)):
            for i2 in range(i1 + 1, len(self.cards)):
                if self.cards[i1].rank == self.cards[i2].rank:
                    pair_count += 1
        return pair_count > 1

    def has_three_of_kind(self):
        """ Returns True if the hand has three cards of the same rank, False otherwise 
        
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,4), Card(2,2), Card(1,6), Card(3,3), Card(2,4), Card(1,1), Card(0,5)])
        >>> hand.has_three_of_kind()
        True
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(3,4), Card(2,2), Card(1,6), Card(3,3), Card(2,4), Card(1,1), Card(0,5)])
        >>> hand.has_three_of_kind()
        False
        """
        # Could also be done with a dictionary keeping track of card occurrences.
        # That would probably be faster with large hands, but this is probably
        # slightly faster given small hands.
        card_count = len(self.cards)
        for i1 in range(card_count - 2):
            for i2 in range(i1 + 1, card_count - 1):
                if self.cards[i1].rank == self.cards[i2].rank:
                    for i3 in range(i2 + 1, card_count):
                        if self.cards[i2].rank == self.cards[i3].rank:
                            return True
        return False

    def has_straight(self):
        """ Returns True if the hand contains five consecutively ranked cards, False otherwise.
        
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,4), Card(2,2), Card(1,6), Card(3,3), Card(2,4), Card(3,1), Card(0,5)])
        >>> hand.has_straight()
        True
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,4), Card(2,2), Card(1,6), Card(3,3), Card(2,7), Card(3,1), Card(0,8)])
        >>> hand.has_straight()
        False
        """
        self.cards.sort(key=lambda c: c.rank)

        # Add a fake '14' card if we have an ace
        if self.cards[0].rank == 1:
            my_cards = self.cards[:]
            my_cards.append(Card(1, 14))
        else:
            my_cards = self.cards
        return PokerHand.__check_for_straight(my_cards)

    def __check_for_straight(cards):
        """ Checks a given list of Cards for a straight.

            Precondition: list must be sorted ascending according to rank.

            cards: a list of Cards
            returns: True if there is a straight, False otherwise.
        """
        card_count = len(cards)
        for i1 in range(card_count - 4):
            card_check_list = [0, 0, 0, 0]
            first_card_rank = cards[i1].rank
            for i_subseq in range(i1 + 1, card_count):
                rank_delta = cards[i_subseq].rank - first_card_rank
                if rank_delta > 4:
                    # Gone reached a card out of the range of a straight
                    break
                if rank_delta > 0:  # Make sure not a duplicate starting card
                    card_check_list[rank_delta - 1] = 1
            if sum(card_check_list) == 4:
                return True  # We found all the cards needed for a straight
        return False


    # NOTE: Both of the following functions could have been done in similar manners, but slightly different methods
    # were employed. I don't know if I picked the fasted method for each or not (and I don't really care to test).

    def has_full_house(self):
        """ Returns True if hand has at least three cards of one rank and at least two of another, False otherwise.

        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,3), Card(2,3), Card(1,6), Card(3,4), Card(2,4), Card(3,1), Card(0,4)])
        >>> hand.has_full_house()
        True
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,3), Card(2,5), Card(1,6), Card(3,4), Card(2,4), Card(3,1), Card(0,4)])
        >>> hand.has_full_house()
        False
        """
        rank_counts = {r: 0 for r in range(13)}
        for card in self.cards:
            rank_counts[card.rank] += 1
        # Check for full house
        counts = list(rank_counts.values())
        counts.sort(reverse=True)
        return counts[0] > 2 and counts[1] > 1

    def has_four_of_kind(self):
        """ Returns True if the hand has four cards of the same suit, False otherwise.

        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,3), Card(2,3), Card(1,3), Card(3,4), Card(2,4), Card(3,3), Card(0,4)])
        >>> hand.has_four_of_kind()
        True
        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,3), Card(2,3), Card(1,2), Card(3,4), Card(2,4), Card(3,3), Card(0,4)])
        >>> hand.has_four_of_kind()
        False
        """
        rank_counts = {r: 0 for r in range(13)}
        for card in self.cards:
            rank_counts[card.rank] += 1
        # Check for four of a kind
        for suit, count in rank_counts.items():
            if count > 3:
                return True
        return False

    def has_straight_flush(self):
        """ Returns True if the deck has five cards that are both of the same suit and of sequential rank, False otherwise.

        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,3), Card(2,3), Card(1,3), Card(3,4), Card(2,4), Card(3,3), Card(0,4)])
        >>> hand.has_straight_flush()
        False

        >>> hand = PokerHand()
        >>> hand.cards.extend([Card(1,10), Card(1,12), Card(1,1), Card(1,13), Card(2,4), Card(3,3), Card(1,11)])
        >>> hand.has_straight_flush()
        True
        """
        cards_by_suit = [list(), list(), list(), list()]
        for card in self.cards:
            cards_by_suit[card.suit].append(card)
        for cards_same_suit in cards_by_suit:
            if len(cards_same_suit) < 5: continue
            cards_same_suit.sort(key=lambda c: c.rank)
            if cards_same_suit[0].rank == 1:
                cards_same_suit.append(Card(cards_same_suit[0].suit,14))
            make_straight = PokerHand.__check_for_straight(cards_same_suit)
            if make_straight:
                return True
        return False

    """
    three of a kind: three cards with the same rank
    straight: five cards with ranks in sequence (aces can be high or low, so Ace-2-3-4-5 is a straight and so is 10-Jack-Queen-King-Ace, but Queen-King-Ace-2-3 is not.)
    flush: five cards with the same suit
    full house: three cards with one rank, two cards with another
    four of a kind: four cards with the same rank
    straight flush"""


if __name__ == '__main__':
    print('Running doctests...')
    doctest.testmod()

Running doctests...
**********************************************************************
File "__main__", line 68, in __main__.PokerHand.has_three_of_kind
Failed example:
    hand.has_three_of_kind()
Expected:
    True
Got:
    False
**********************************************************************
1 items had failures:
   1 of   6 in __main__.PokerHand.has_three_of_kind
***Test Failed*** 1 failures.


## Reading Journal feedback

[Please complete this short survey](https://docs.google.com/forms/d/e/1FAIpQLScQekhUrf6YYjpfQiAAbavLIA-IJklv_PX1BWbGgxj7JPolmw/viewform?c=0&w=1)

If you have any comments on this Reading Journal, feel free to leave them in the survey linked above. This could include suggestions to improve the exercises, topics you'd like to see covered in class next time, or other feedback.

If you have Python questions or run into problems while completing the reading, you should post them to Piazza instead so you can get a quick response before your journal is submitted.